In [ ]:
# Create venv 'python3 -m venv .venv'
# Activate venv 'source .venv/bin/activate'
# Select the correct kernel.
import json
import requests
import pandas as pd
import cufflinks as cf
import plotly.express as px
import plotly.offline as pyo
import plotly.io as pio
from IPython.display import display, HTML

pyo.init_notebook_mode(connected=True)

enigh = '/Users/emanuelalarcon/Documents/Bases de datos/ENIGH/'

viviendas = pd.read_csv(enigh + 'viviendas.csv')
ingresos = pd.read_csv(enigh + 'ingresos.csv')
hogares = pd.read_csv(enigh + 'hogares.csv')

## Funciones y diccionarios

In [2]:
def asignarTipoAdqui(row):
    if row == '1':
        return 'La compró hecha'
    elif row == '2':
        return 'La mandó construir'
    elif row == '3':
        return 'La construyó él mismo'
    elif row == '4':
        return 'La obtuvo de otra manera'
    else:
        return 'No es propia'

In [3]:
claves_inegi = {
    '01': 'Aguascalientes',
    '02': 'Baja California',
    '03': 'Baja California Sur',
    '04': 'Campeche',
    '05': 'Coahuila',
    '06': 'Colima',
    '07': 'Chiapas',
    '08': 'Chihuahua',
    '09': 'Ciudad de México',
    '10': 'Durango',
    '11': 'Guanajuato',
    '12': 'Guerrero',
    '13': 'Hidalgo',
    '14': 'Jalisco',
    '15': 'México',
    '16': 'Michoacán',
    '17': 'Morelos',
    '18': 'Nayarit',
    '19': 'Nuevo León',
    '20': 'Oaxaca',
    '21': 'Puebla',
    '22': 'Querétaro',
    '23': 'Quintana Roo',
    '24': 'San Luis Potosí',
    '25': 'Sinaloa',
    '26': 'Sonora',
    '27': 'Tabasco',
    '28': 'Tamaulipas',
    '29': 'Tlaxcala',
    '30': 'Veracruz',
    '31': 'Yucatán',
    '32': 'Zacatecas'
}

## Tipo de adquisición

In [ ]:
dfTipoAdqui = viviendas[['folioviv', 'tipo_adqui', 'factor']]
dfTipoAdqui

In [ ]:
dfTipoAdqui['tipo_adqui'] = dfTipoAdqui.tipo_adqui.apply(asignarTipoAdqui)
dfTipoAdqui

In [ ]:
dfTipoAdquiResumida = dfTipoAdqui.groupby('tipo_adqui').agg(viviendas=('factor', 'sum')).reset_index()
dfTipoAdquiResumida

In [ ]:
totalViviendas = dfTipoAdquiResumida['viviendas'].sum()
print(totalViviendas)

In [ ]:
dfTipoAdquiResumida['porcentaje'] = (dfTipoAdquiResumida['viviendas'] / totalViviendas * 100).round(1)
dfTipoAdquiResumida

In [ ]:
dfTipoAdquiResumida = dfTipoAdquiResumida.sort_values(by='porcentaje', ascending=True)
dfTipoAdquiResumida

In [ ]:
fig1 = px.bar(dfTipoAdquiResumida,
             title='Tipo de Adquisición de la Vivienda en México como Porcentaje del Total',
             x='porcentaje',
             y='tipo_adqui',
             #color_discrete_sequence=['#bf282a']
             labels={
                 'porcentaje' : 'Porcentaje de viviendas',
                 'tipo_adqui' : 'Tipo de adquisición'
             }
             )
fig1.update_layout(
    xaxis_title='Porcentaje',
    yaxis_title='Tipo de Adquisición',
    annotations=[
        dict(
            x=1,
            y=-0.15,
            showarrow=False,
            text='Fuente: Elaboración propia con datos de la ENIGH, 2022',
            xref='paper',
            yref='paper',
            xanchor='right',
            yanchor='top',
            font=dict(size=12, color='gray')
        )
    ]
)

fig1.show()

## Renta como porcentaje del ingreso

In [ ]:
dfRentaHogar = viviendas[['folioviv', 'renta', 'factor']]
dfRentaHogar

In [ ]:
dfIngresoHogar = ingresos[['folioviv', 'foliohog', 'ing_tri']]
dfIngresoHogar

In [ ]:
dfRentaIngreso = pd.merge(dfRentaHogar, dfIngresoHogar, on='folioviv', how='inner')
dfRentaIngreso

In [ ]:
dfRentaIngresoTotal = dfRentaIngreso.groupby(['folioviv', 'foliohog']).agg(ingreso_total_hogar_tri=('ing_tri', 'sum')).reset_index()
dfRentaIngresoTotal

In [ ]:
dfRentaIngresoTotal['ingreso_total_hogar_men'] = (dfRentaIngresoTotal['ingreso_total_hogar_tri'] / 3).round(2)
dfRentaIngresoTotal

In [ ]:
dfRentaHogar = pd.merge(dfRentaHogar, dfRentaIngresoTotal[['folioviv', 'foliohog' ,'ingreso_total_hogar_men']], on='folioviv', how='inner')
dfRentaHogar

In [ ]:
dfRentaHogar['renta'] = dfRentaHogar['renta'].replace(" ", 0)
dfRentaHogar

In [ ]:
dfRentaHogar.dtypes

In [ ]:
dfRentaHogar['renta'] = pd.to_numeric(dfRentaHogar['renta'])
dfRentaHogar

In [ ]:
dfRentaHogar.dtypes

In [ ]:
dfRentaHogar = dfRentaHogar[(dfRentaHogar['renta'] > 0)]
dfRentaHogar

In [ ]:
dfRentaHogar['num_hogares'] = dfRentaHogar.groupby('folioviv')['foliohog'].transform('count')
dfRentaHogar

In [ ]:
dfRentaHogar[dfRentaHogar['num_hogares'] > 1]

In [ ]:
dfRentaHogar = dfRentaHogar[['folioviv', 'foliohog', 'renta', 'ingreso_total_hogar_men', 'num_hogares', 'factor']]
dfRentaHogar

In [ ]:
dfRentaHogar['renta_hogar_men'] = (dfRentaHogar['renta'] / dfRentaHogar['num_hogares']).round(2)
dfRentaHogar

In [ ]:
dfRentaHogar['porcentaje_en_renta'] = (dfRentaHogar['renta_hogar_men'] / dfRentaHogar['ingreso_total_hogar_men'] * 100).round(2)
dfRentaHogar

In [ ]:
dfRentaHogar['quintil'] = pd.qcut(dfRentaHogar['ingreso_total_hogar_men'], q=5, labels=False) + 1
dfRentaHogar

In [ ]:
dfRentaHogarPlot = dfRentaHogar[['porcentaje_en_renta', 'quintil', 'factor']]
dfRentaHogarPlot

In [ ]:
dfRentaHogarPlot = dfRentaHogarPlot.loc[dfRentaHogarPlot.index.repeat(dfRentaHogarPlot['factor'].round())].reset_index(drop=True)
dfRentaHogarPlot

In [ ]:
dfRentaHogarPlot = dfRentaHogarPlot.drop(columns='factor')
dfRentaHogarPlot

In [ ]:
fig2 = px.box(
    dfRentaHogarPlot,
    x='quintil',
    y='porcentaje_en_renta',
    title='Gasto en Arrendamiento como Porcentaje del Ingreso por Decil en México, Mensual'
)
fig2.update_layout(
    xaxis_title='Quintil',
    yaxis_title='Porcentaje del ingreso destinado al arrendamiento',
    annotations=[
        dict(
            x=1,
            y=-0.15,
            showarrow=False,
            text='Fuente: Elaboración propia con datos de la ENIGH, 2022',
            xref='paper',
            yref='paper',
            xanchor='right',
            yanchor='top',
            font=dict(size=12, color='gray')
        )
    ]
)
fig2.update_yaxes(range=[0, 100])

fig2.show()

## Arrendamiento medio por estado de la República

In [ ]:
dfEntidadVivienda = hogares[['folioviv', 'entidad']]
dfEntidadVivienda

In [ ]:
dfEntidadVivienda.dtypes

In [ ]:
dfEntidadVivienda['entidad'] = dfEntidadVivienda['entidad'].astype(str).str.zfill(2)
dfEntidadVivienda

In [ ]:
dfEntidadVivienda['entidad'] = dfEntidadVivienda['entidad'].map(claves_inegi)
dfEntidadVivienda

In [ ]:
valoresUnicosFoioViv = viviendas['folioviv'].unique()
valoresUnicosFoioViv.size

In [ ]:
dfArrendamientoEntidad = pd.DataFrame({'folioviv' : valoresUnicosFoioViv})
dfArrendamientoEntidad

In [ ]:
dfRentaViv = viviendas[['folioviv', 'renta', 'factor']]
dfRentaViv

In [ ]:
dfArrendamientoEntidad = pd.merge(dfArrendamientoEntidad, dfRentaViv, on='folioviv', how='inner')
dfArrendamientoEntidad

In [ ]:
dfArrendamientoEntidad['folioviv'].unique

In [ ]:
dfEntidadVivienda = dfEntidadVivienda.drop_duplicates(subset='folioviv')
dfEntidadVivienda

In [ ]:
dfEntidadVivienda

In [ ]:
dfArrendamientoEntidad = pd.merge(dfArrendamientoEntidad, dfEntidadVivienda, on='folioviv', how='left')
dfArrendamientoEntidad

In [ ]:
dfArrendamientoEntidad.dtypes

In [ ]:
dfArrendamientoEntidad['renta'] = dfArrendamientoEntidad['renta'].astype(int)
dfArrendamientoEntidad

In [ ]:
dfArrendamientoEntidad = dfArrendamientoEntidad[dfArrendamientoEntidad['renta'] > 0]
dfArrendamientoEntidad

In [ ]:
dfArrendamientoEntidad['renta_por_factor'] = dfArrendamientoEntidad['renta'] * dfArrendamientoEntidad['factor']
dfArrendamientoEntidad

In [ ]:
dfArrendamientoEntidad.sample(15)

In [ ]:
factoresPorEstado = dfArrendamientoEntidad.groupby('entidad')['factor'].sum().reset_index()
factoresPorEstado

In [ ]:
test = dfArrendamientoEntidad[dfArrendamientoEntidad['entidad'] == 'Ciudad de México']['factor'].sum()
print(test)

In [ ]:
dfArrendamientoEntidad = dfArrendamientoEntidad.groupby('entidad')['renta_por_factor'].sum().reset_index()
dfArrendamientoEntidad

In [ ]:
dfArrendamientoEntidad = pd.merge(dfArrendamientoEntidad, factoresPorEstado, on='entidad', how='inner')
dfArrendamientoEntidad

In [ ]:
dfArrendamientoEntidad['arrendamiento_medio'] = (dfArrendamientoEntidad['renta_por_factor'] / dfArrendamientoEntidad['factor']).round(2)
dfArrendamientoEntidad

In [159]:
geojson_url = 'https://raw.githubusercontent.com/angelnmara/geojson/master/mexicoHigh.json'
geojson = requests.get(geojson_url).json()

In [ ]:
fig3 = px.choropleth(dfArrendamientoEntidad,
                    geojson=geojson,
                    locations='entidad',
                    featureidkey="properties.name",
                    color='arrendamiento_medio',
                    color_continuous_scale="burg",
                    title='Arrendamiento Medio por Entidad Federativa',
                    )

fig3.update_geos(fitbounds="locations", visible=False)
fig3.update_traces(marker_line_width=0.1)

fig3.update_layout(
    title_font=dict(color='rgb(47,62,92)'),
    geo_bgcolor='rgba(0,0,0,0)',
    margin={"r":0,"t":50,"l":0,"b":0},
    coloraxis_colorbar=dict(
        title="Media",
        lenmode="fraction", len=0.75
    ),
    width=1144,
    height=384,
    annotations=[
        dict(
            x=1,
            y=1,
            showarrow=False,
            text='Fuente: Elaboración propia con datos de la ENIGH, 2022',
            xref='paper',
            yref='paper',
            xanchor='right',
            yanchor='top',
            font=dict(size=12, color='gray')
        )
    ]
)

fig3.show()